# TM10007 Assignment template

In [79]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [80]:
# Data loading functions. Uncomment the one you want to use
# Import other classifiers you plan to use
from worclipo.load_data import load_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from scipy.stats import ttest_ind

data = load_data()
print(f"The number of samples: {len(data.index)}")
print(f"The number of columns: {len(data.columns)}")
print(f"The number of liposarcoma in the dataset: {len(data[data['label'] == 'liposarcoma'])}")
print(f"The number of lipoma in the dataset: {len(data[data['label'] == 'lipoma'])}")

The number of samples: 115
The number of columns: 494
The number of liposarcoma in the dataset: 58
The number of lipoma in the dataset: 57


Now we know the number of samples, colums and labels. The next step is to look for missing data. 


In [81]:
def split_data(data):

    # Separate features and target variable
    x = data.drop(['label'], axis=1)
    y = data['label']
    
    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test


# Call the function
x_train, x_test, y_train, y_test = split_data(data)

# Optionally, print the sizes of the splits to verify
print("Training set size:", x_train.shape)
print("Testing set size:", x_test.shape)

Training set size: (92, 493)
Testing set size: (23, 493)


In [82]:
def find_zeros_in_data(data):
    # Maak een kopie van de data om de originele data niet te wijzigen
    data_copy = data.copy()
    
    # Vervang waarden die gelijk zijn aan 0 met NaN om ze gemakkelijk te tellen
    data_copy = data_copy.replace(0, np.nan)
    
    # Tel het aantal NaNs in elke kolom, wat overeenkomt met het originele aantal 0's
    zeros_count = data_copy.isna().sum()
    
    # Print het aantal 0's gevonden in elke kolom
    print("Aantal 0's per feature:\n", zeros_count)
    
    # Return het aantal 0's per feature voor verdere analyse indien nodig
    return zeros_count

# Roep de functie aan met je dataset 'data'
zeros_count = find_zeros_in_data(data)


Aantal 0's per feature:
 label                                                       0
PREDICT_original_sf_compactness_avg_2.5D                    0
PREDICT_original_sf_compactness_std_2.5D                    0
PREDICT_original_sf_rad_dist_avg_2.5D                       0
PREDICT_original_sf_rad_dist_std_2.5D                       0
                                                         ... 
PREDICT_original_phasef_phasesym_peak_position_WL3_N5     115
PREDICT_original_phasef_phasesym_range_WL3_N5               0
PREDICT_original_phasef_phasesym_energy_WL3_N5              0
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5     12
PREDICT_original_phasef_phasesym_entropy_WL3_N5             0
Length: 494, dtype: int64


Feature normalization

In [83]:
def scaling_data(x_train,x_test):
    scaler = preprocessing.StandardScaler().fit(x_train)

    # Applying scaler to train and test set
    X_train_scaled = scaler.transform(x_train)
    X_test_scaled = scaler.transform(x_test)

    return X_train_scaled, X_test_scaled


Feature selection 

In [111]:
#define samples
lipoma = data[data['label']=='lipoma']
liposarcoma = data[data['label']=='liposarcoma']

Ttest_indResult = pd.DataFrame(columns=['t_statistic', 'p_value'])
for feature in data.columns:
    if feature != 'label':  # Exclude the label column
        t_statistic, p_value = ttest_ind(lipoma[feature], liposarcoma[feature])
        feature_result = pd.DataFrame({'t_statistic': t_statistic, 'p_value': p_value}, index = [feature])
        Ttest_indResult = pd.concat([Ttest_indResult, feature_result])


absolute_features = Ttest_indResult['t_statistic'].abs()
best_festures = absolute_features.nlargest(5)


C:\Users\31642\AppData\Local\Temp\ipykernel_12572\3512054077.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Ttest_indResult = pd.concat([Ttest_indResult, feature_result])
c:\Users\31642\miniconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
c:\Users\31642\miniconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*

                                                    t_statistic   p_value
PREDICT_original_sf_compactness_avg_2.5D               0.690264  0.491443
PREDICT_original_sf_compactness_std_2.5D              -3.058812  0.002775
PREDICT_original_sf_rad_dist_avg_2.5D                 -2.161416  0.032773
PREDICT_original_sf_rad_dist_std_2.5D                 -0.618317  0.537611
PREDICT_original_sf_roughness_avg_2.5D                 0.632823  0.528128
...                                                         ...       ...
PREDICT_original_phasef_phasesym_peak_position_...          NaN       NaN
PREDICT_original_phasef_phasesym_range_WL3_N5          2.660891  0.008928
PREDICT_original_phasef_phasesym_energy_WL3_N5        -2.779793  0.006373
PREDICT_original_phasef_phasesym_quartile_range...     3.273034  0.001412
PREDICT_original_phasef_phasesym_entropy_WL3_N5       -2.791192  0.006167

[493 rows x 2 columns]
PREDICT_original_tf_Gabor_mean_F0.2_A0.0        4.360695
PREDICT_original_tf_Gabor_mean_